In [5]:
# ! pip install albumentations
# ! pip install --upgrade typing_extensions 

# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# tensorflow 2.4.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.4.0 which is incompatible.
# tensorflow-gpu 2.4.0 requires typing-extensions~=3.7.4, but you have typing-extensions 4.4.0 which is incompatible.
# Successfully installed typing_extensions-4.4.0

## generate augmented images

In [13]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random


DIR_DATASET = '/home/young/hdd1'

def load_imgpath(imagefilelist, category, n_sample=20):
    img_names = []
    with open(imagefilelist, 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = line.strip().split(' ')
            if int(data[1]) == d_name2idx[category]:
                img_names.append(data[0])
                
    random.seed(1) #15
    img_names = random.sample(img_names, n_sample)
    img_names = [im.replace('/data/datasets', DIR_DATASET) for im in img_names]
    return img_names


import matplotlib.pyplot as plt
from torchvision.utils import save_image

def load_and_transform(img_names, category, expname, visualize=False, save_dir=None):
    
    orig_trans = A.Compose([
                    A.Resize(height=256, width=256),
                    A.CenterCrop(height=224, width=224),
#                     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                    A.ToFloat(),
                    ToTensorV2()])

    if expname == 'exp_position':
        tran_trans = A.Compose([
                        A.Resize(height=256, width=256),
                        A.CenterCrop(height=224, width=224),
                        A.Affine(translate_percent=(-0.5,0.5),  p=1.0),
    #                     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        A.ToFloat(), # if using normalize, float is not necessary
                        ToTensorV2()])  
        
    elif expname == 'exp_size':

        tran_trans = A.Compose([
                        A.Resize(height=256, width=256),
                        A.CenterCrop(height=224, width=224),
                        A.Affine(scale=(0.5, 1.5),  p=1.0),
    #                     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        A.ToFloat(), # if using normalize, float is not necessary
                        ToTensorV2()])

        
        
    elif expname == 'exp_stats':
        
        tran_trans = A.Compose([
                        A.Resize(height=256, width=256),
                        A.CenterCrop(height=224, width=224),
                        A.Equalize(mode='cv', p=1.0),
#                         A.ColorJitter(p=1.0),
#                         A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        A.ToFloat(), # if using normalize, float is not necessary
                        ToTensorV2()])  
        
    elif expname == 'exp_sf':     
        tran_trans = A.Compose([
                        A.Resize(height=256, width=256),
                        A.CenterCrop(height=224, width=224),
#                         A.Sharpen(p=1.0),
                        A.Blur(blur_limit=3, p=1.0),
    #                     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                        A.ToFloat(), # if using normalize, float is not necessary
                        ToTensorV2()])  
    else:
        raise NotImplementedError
    
    
    orig_imgs, trans_imgs =[], []
    for img in img_names:
        img = np.array(Image.open(img).convert('RGB'))
        if img is None:
            print(im_name)
            
        oimg = orig_trans(image=img)['image']
        timg = tran_trans(image=img)['image']
        
        orig_imgs.append(oimg)
        trans_imgs.append(timg)
                
            
    if visualize:
        for i, (oimg, timg) in enumerate(zip(orig_imgs, trans_imgs)):
            fig, (ax1, ax2) = plt.subplots(1, 2)    
            ax1.imshow(oimg.cpu().numpy().transpose((1, 2, 0)))
            ax2.imshow(timg.cpu().numpy().transpose((1, 2, 0)))
            plt.show()

    if save_dir is not None:

        for i, (oimg, timg) in enumerate(zip(orig_imgs, trans_imgs)):
            path_orig = os.path.join(save_dir, expname, f'{category}_orig', f'{category}_{i}.jpeg')
            path_transform =os.path.join(save_dir, expname, f'{category}_tran', f'{category}_{i}.jpeg')

            if not os.path.exists(os.path.dirname(path_orig)):
                os.makedirs(os.path.dirname(path_orig))
            if not os.path.exists(os.path.dirname(path_transform)):
                os.makedirs(os.path.dirname(path_transform))
            
            save_image(oimg, path_orig)
            save_image(timg, path_transform)
        print('images saved to disk')

In [14]:
# generate images
imagefilelist = './data/imagenet-16/imagenet-16-val_list.txt'
# expname = 'exp_size' #'exp_position', 'exp_size', 'exp_stats'
for expname in ['exp_size', 'exp_position', 'exp_stats', 'exp_sf']:
    for category in d_name2idx.keys():
        print(category)
        img_names = load_imgpath(imagefilelist, category)
        load_and_transform(img_names, category, expname, visualize=False, save_dir='./data/imagenet-16/') 
    #     load_and_transform(img_names, category, expname, visualize=True) 


elephant
images saved to disk
chair
images saved to disk
bicycle
images saved to disk
car
images saved to disk
bird
images saved to disk
truck
images saved to disk
dog
images saved to disk
boat
images saved to disk
keyboard
images saved to disk
bottle
images saved to disk
cat
images saved to disk
knife
images saved to disk
airplane
images saved to disk
clock
images saved to disk
bear
images saved to disk
oven
images saved to disk
elephant
images saved to disk
chair
images saved to disk
bicycle
images saved to disk
car
images saved to disk
bird
images saved to disk
truck
images saved to disk
dog
images saved to disk
boat
images saved to disk
keyboard
images saved to disk
bottle
images saved to disk
cat
images saved to disk
knife
images saved to disk
airplane
images saved to disk
clock
images saved to disk
bear
images saved to disk
oven
images saved to disk
elephant
images saved to disk
chair
images saved to disk
bicycle
images saved to disk
car
images saved to disk
bird
images saved to 

In [15]:
# generate datasource
import os
import argparse

DIR_DATASOURCE= './data/imagenet-16'
    
for expname in ['exp_position', 'exp_sf', 'exp_size', 'exp_stats']:
    
    FILENAME = 'imagenet-16-val-aug'
    forig = open(os.path.join(DIR_DATASOURCE, FILENAME + '_'+ expname +'_original_list.txt'), 'w')
    ftran = open(os.path.join(DIR_DATASOURCE, FILENAME + '_'+ expname +'_transform_list.txt'), 'w')

    exp_folder = os.path.join(DIR_DATASOURCE, expname)
    folders = os.listdir(exp_folder)
    for i, folder in enumerate(folders):
        folder_path = os.path.join(exp_folder, folder)
        files = os.listdir(folder_path)
        files = [f for f in files if not f.startswith('.')]
        if folder.split('_')[1] == 'orig':
            for file in files:
                forig.write('{} {}\n'.format(os.path.join(folder_path, file), folder.split('_')[0]))
        if folder.split('_')[1] == 'tran':
            for file in files:
                ftran.write('{} {}\n'.format(os.path.join(folder_path, file), folder.split('_')[0]))
                
    forig.close()
    ftran.close()
    
print('lists were generated')

lists were generated
